Step 1: Preparation 

Before you begin, make sure that you have set the runtime type to GPU  (Hardware acclerator: GPU).

In [ ]:
ROOTPATH="/content/ailab/" # root dir (a constant)
ModelPath=ROOTPATH+"Real-CUGAN/model/" # model dir
PendingPath=ROOTPATH+"Real-CUGAN/pending/" # input dir
FinishPath=ROOTPATH+"Real-CUGAN/finish/" # output dir
ModelName="up2x-latest-no-denoise.pth" # default model
Tile=4 #{0,1,2,3,4,auto}; the larger the number, the smaller the memory consumption

# initialize environment
!pip install torch opencv-python
!git clone https://github.com/bilibili/ailab.git
from google.colab import drive
drive.mount('/content/gdrive')

Step 2: Download model files from [here](https://drive.google.com/drive/folders/1UFgpV14uEAcgYvVw0fJuajzy1k7JIz6H) and save them - in a folder called `updated_weights` - under your Google Drive's root folder.

Step 3: Put/upload your image(s) under `/content/aliab/Real-CUGAN/pending`.

Then, run the following and choose the model you want.

In [ ]:
import os
os.mkdir(ModelPath)
os.mkdir(PendingPath)
os.mkdir(FinishPath)
!cp -r /content/gdrive/MyDrive/updated_weights/* /content/ailab/Real-CUGAN/model/
fileNames = os.listdir(PendingPath)
print("Pending images:")
for i in fileNames:
  print("\t"+i)
fileNames = os.listdir(ModelPath)
print("Model files available:")
for idx, i in enumerate(fileNames):
  print(f"{idx+1}. \t {i}")
choice = int(input("Select model (leave blank for default)："))
if choice:
  ModelName=fileNames[choice-1]
Amplification=ModelName[2] # amplifying ratio
if (not os.path.isfile(ModelPath+ModelName)):
  print("Warning: selected model file does not exist")

Step 4: Run the processing script.

In [ ]:
import sys
sys.path.append("/content/ailab/Real-CUGAN")
import matplotlib.pyplot as plt
%matplotlib inline
import torch
from torch import nn as nn
from torch.nn import functional as F
import os,sys,cv2
import numpy as np
from upcunet_v3 import RealWaifuUpScaler
from time import time as ttime  
fileNames = os.listdir(PendingPath)
print(f"using model {ModelPath+ModelName}")
upscaler = RealWaifuUpScaler(2, ModelPath+ModelName, half=True, device="cuda:0")
t0 = ttime()
for i in fileNames:
  torch.cuda.empty_cache()
  try:
    img = cv2.imread(PendingPath+i)[:, :, [2, 1, 0]]
    result = upscaler(img,tile_mode=5,cache_mode=2,alpha=1)
    cv2.imwrite(FinishPath+i,result[:, :, ::-1])
  except RuntimeError as e:
    print (i+" FAILED")
    print (e)
  else:
    print(i+" DONE")
    os.remove(PendingPath+i)
t1 = ttime()
print("time_spent", t1 - t0)

Generated images should be under /content/aliab/Real-CUGAN/finish.